# Ch 8  Data Wrangling Join Combine and Reshape 数据规整 连接 联合 与重塑

In [5]:
import pandas as pd
import numpy as np

# 8.1 分层索引：Hierarchical Indexing 

In [15]:
#分层索引是pandas重要特性，允许在一个轴向上拥有多个索引层级。
#分层索引提供了一种在更低维度处理更高维度数据的方式。
#先创建一个series，以列表的列表作为索引：

data = pd.Series(np.random.randn(9),
                index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data
# a  1   -1.305418
#    2    1.776525
#    3   -0.201306
# b  1   -0.670462
#    3   -0.233106
# c  1   -0.837201
#    2   -0.001711
# d  2   -0.729868
#    3    1.570326
# dtype: float64

#此处看到的是以multiIndex作为索引的series的美化视图。索引中的间隙，表示直接使用上面的标签。
data.index
# MultiIndex([('a', 1),
#             ('a', 2),
#             ('a', 3),
#             ('b', 1),
#             ('b', 3),
#             ('c', 1),
#             ('c', 2),
#             ('d', 2),
#             ('d', 3)],
#            )

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [33]:
#通过分层索引对象，也可以成为部分索引，允许简洁地选择出数据的子集：
data['b']
# 1   -0.973907
# 3   -0.593977
# dtype: float64
data['b':'c']
# b  1   -0.973907
#    3   -0.593977
# c  1    0.636276
#    2    1.477976
# dtype: float64
data.loc[['b','d']]
# b  1   -0.973907
#    3   -0.593977
# d  2   -1.099617
#    3    0.175689
# dtype: float64

#在内部层级中选择也可以：
data.loc[:, 2]     #这从DataFrame对象 data 中选择所有行的第2列数据。?
# a   -0.680452
# c    1.477976
# d   -1.099617
# dtype: float64



a   -0.680452
c    1.477976
d   -1.099617
dtype: float64

In [34]:
#unstack：unstack() 是一个用于将多层索引的数据进行重塑的方法。
#如果DataFrame对象具有层次化的列索引（MultiIndex），则可以使用 unstack() 方法将其中一个或多个层次的列索引转换为行索引，
#从而得到一个更加扁平化的表格形式。

#分层索引在重塑数据和数组透视表等分组操作中很重要。
#可以用unstack方法，在data frame中重新排列：
data.unstack()
# 1	2	3
# a	-1.815016	-0.680452	0.069912
# b	-0.973907	NaN	-0.593977
# c	0.636276	1.477976	NaN
# d	NaN	-1.099617	0.175689

#unstack的反操作是stack：
data.unstack().stack()
# a  1   -1.815016
#    2   -0.680452
#    3    0.069912
# b  1   -0.973907
#    3   -0.593977
# c  1    0.636276
#    2    1.477976
# d  2   -1.099617
#    3    0.175689
# dtype: float64

a  1   -1.815016
   2   -0.680452
   3    0.069912
b  1   -0.973907
   3   -0.593977
c  1    0.636276
   2    1.477976
d  2   -1.099617
   3    0.175689
dtype: float64

In [44]:
#在data frame中，每个轴都可以拥有分层索引：
#先创建一个4*3的表，纵轴定为a,b，每个字母下设12，横轴定为Ohio, Colorado, Ohio下设Green, Red
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns = [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame
# 	Ohio	Colorado
# Green	Red	Green
# a	1	0	1	2
# 2	3	4	5
# b	1	6	7	8
# 2	9	10	11

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [52]:
#分层的层级可以有名称（字符串或python对象）。如果层级有名称，名称会在控制台输出中显示。

#将a,b;1,2命名为key；将ohio, colorado; green, red, green命名为state, color
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame
# 	state	Ohio	Colorado
# color	Green	Red	Green
# key1	key2			
# a	1	0	1	2
# 2	3	4	5
# b	1	6	7	8
# 2	9	10	11

#通过部分列索引，可以选出列中的组：
frame['Ohio']
# 	color	Green	Red
# key1	key2		
# a	1	0	1
# 2	3	4
# b	1	6	7
# 2	9	10

#一个multiIndex的对象可以使用自身的构造函数创建并复用。带有层级的data frame的列可以这样创建：
#MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']], names = ['state', 'color'])

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

## 8.1.1 重排序和层次排序 Reordering and Sorting Levels

In [58]:
#需要重新排列轴上的层级顺序，或按照特定层级的值，对数据排序。
#swaplevel接收两个层级序号或层级名称。返回一个进行了层级变更的新对象。

frame.swaplevel('key1', 'key2')

#sort_index只能在单一层级上，对数据排序。
#进行层级变换时，使用sort_index使得结果按照层级进行字典排序也常见：
frame.sort_index(level = 1)                                    #表示根据索引的第一层级进行排序。
# 	state	Ohio	Colorado
# color	Green	Red	Green
# key1	key2			
# a	1	0	1	2
# b	1	6	7	8
# a	2	3	4	5
# b	2	9	10	11

frame.swaplevel(0, 1).sort_index(level = 0)              #交换DataFrame对象 frame 的两个层级的顺序，并在第一个层级上进行排序。
# # state	Ohio	Colorado
# # color	Green	Red	Green
# # key2	key1			
# # 1	a	0	1	2
# # b	6	7	8
# # 2	a	3	4	5
# # b	9	10	11

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 8.1.2 按层级进行汇总统计 Summary Statistics by Level

In [63]:
#dataframe和series中很多描述性和汇总性统计有一个level选项，通过level选项可以指定再某个特定的轴上进行聚合。
#可以按照层级在行或列聚合：

frame.sum(level = 'key2')   #以key2为groupby, 内部进行sum
# state	Ohio	Colorado
# color	Green	Red	Green
# key2			
# 1	6	8	10
# 2	12	14	16

frame.sum(level = 'color', axis = 1)  #以color为group by, 内部进行sum
# color	Green	Red
# key1	key2		
# a	1	2	1
# 2	8	4
# b	1	14	7
# 2	20	10

C:\Users\miran\AppData\Local\Temp\ipykernel_76136\1706064382.py:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level = 'key2')   #以key2为groupby, 内部进行sum
C:\Users\miran\AppData\Local\Temp\ipykernel_76136\1706064382.py:11: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level = 'color', axis = 1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 8.1.3 使用data frame的列进行索引 Indexing with a DataFrame's columns

In [76]:
#当想将行索引移动到data frame中。

frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 
                     'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})

frame
# a	b	c	d
# 0	0	7	one	0
# 1	1	6	one	1
# 2	2	5	one	2
# 3	3	4	two	0
# 4	4	3	two	1
# 5	5	2	two	2
# 6	6	1	two	3

#data frame的set_index会生成一个新的data frame, 新df使用一个或多个列作为索引：
frame2 = frame.set_index(['c', 'd'])                            #把c,d 从列名，变为index行名
frame2
# 		a	b
# c	d		
# one	0	0	7
# 1	1	6
# 2	2	5
# two	0	3	4
# 1	4	3
# 2	5	2
# 3	6	1

#默认情况，这些列会从df中移除，也可以将其留在data frame中：
frame.set_index(['c', 'd'], drop = False)                  #纵列的cd仍然保留，也出现在行名中
# a	b	c	d
# c	d				
# one	0	0	7	one	0
# 1	1	6	one	1
# 2	2	5	one	2
# two	0	3	4	two	0
# 1	4	3	two	1
# 2	5	2	two	2
# 3	6	1	two	3

#reset_index是set_index 反操作，分层索引的行会被返回移动到原来的列中：
frame2.reset_index()
# c	d	a	b
# 0	one	0	0	7
# 1	one	1	1	6
# 2	one	2	2	5
# 3	two	0	3	4
# 4	two	1	4	3
# 5	two	2	5	2
# 6	two	3	6	1

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 联合与合并数据库 Combining and Merging Datasets

## 8.2.1 数据库风格的data frame连接 Database-Style DataFrame Joins

In [86]:
#合并或连接操作通过一个或多个键连接行，来联合数据集。
#pandas中的merge用于将各种join应用于数据

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 
                   'data2': range(3)})

df1
# key	data1
# 0	b	0
# 1	b	1
# 2	a	2
# 3	c	3
# 4	a	4
# 5	a	5
# 6	b	6
df2
# key	data2
# 0	a	0
# 1	b	1
# 2	d	2

#这是多对一的例子：df1的数据有多个行的标签为a,b；而df2在key列每个值仅有一行
#调用merge处理获得的对象：找到两个DataFrame中匹配的列（或索引）并将它们对应的行合并在一起
#因为没有指定再哪一列上进行连接，如果连接的键信息没有指定，merge会自动将重叠列名作为连接的键。
pd.merge(df1, df2)
# key	data1	data2
# 0	b	0	1
# 1	b	1	1
# 2	b	6	1
# 3	a	2	0
# 4	a	4	0
# 5	a	5	0

#如果每个对象的列名是不同的，可以分别制定列名：
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')  #左边DataFrame的 'lkey' 列和右边DataFrame的 'rkey' 列上进行匹配
# lkey	data1	rkey	data2
# 0	b	0	b	1
# 1	b	1	b	1
# 2	b	6	b	1
# 3	a	2	a	0
# 4	a	4	a	0
# 5	a	5	a	0     

#因为结果中缺少'c', 'd'的值，以及相关的数据。
#默认情况下，merge做的是内连接：inner join；其余选项为'left', 'right', 'outer'
pd.merge(df1, df2, how = 'outer')
# 	key	data1	data2
# 0	b	0.0	1.0
# 1	b	1.0	1.0
# 2	b	6.0	1.0
# 3	a	2.0	0.0
# 4	a	4.0	0.0
# 5	a	5.0	0.0
# 6	c	3.0	NaN
# 7	d	NaN	2.0

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [104]:
#多对多的合并，有明确的行为：

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 
                   'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                   'data2': range(5)})
df1
# key	data1
# 0	b	0
# 1	b	1
# 2	a	2
# 3	c	3
# 4	a	4
# 5	b	5
df2
# key	data2
# 0	a	0
# 1	b	1
# 2	a	2
# 3	b	3
# 4	d	4

pd.merge(df1, df2, on = 'key', how = 'left')
# key	data1	data2
# 0	b	0	1.0
# 1	b	0	3.0
# 2	b	1	1.0
# 3	b	1	3.0
# 4	a	2	0.0
# 5	a	2	2.0
# 6	c	3	NaN
# 7	a	4	0.0
# 8	a	4	2.0
# 9	b	5	1.0
# 10	b	5	3.0

#多对多连接是行的笛卡尔积，由于在左边的data frame中有3个'b'行
#在右边有两行，因此在结果中有6个'b'行

pd.merge(df1, df2, how = 'inner')
# 	key	data1	data2
# 0	b	0	1
# 1	b	0	3
# 2	b	1	1
# 3	b	1	3
# 4	b	5	1
# 5	b	5	3
# 6	a	2	0
# 7	a	2	2
# 8	a	4	0
# 9	a	4	2

#on = ['key1', 'key2']：多个键进行合并时，传入一个列名的列表：
df3 = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

df4 = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
df3
# 	key1	key2	lval
# 0	foo	one	1
# 1	foo	two	2
# 2	bar	one	3
df4
# key1	key2	rval
# 0	foo	one	4
# 1	foo	one	5
# 2	bar	one	6
# 3	bar	two	7

pd.merge(df3, df4, on = ['key1', 'key2'], how = 'outer')
# # key1	key2	lval	rval
# # 0	foo	one	1.0	4.0
# # 1	foo	one	1.0	5.0
# # 2	foo	two	2.0	NaN
# # 3	bar	one	3.0	6.0
# # 4	bar	two	NaN	7.0

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [107]:
#merge有个suffixes后缀，用于在左右两边data frame对象的重叠列名后添加字符串：
#eg. key2_x ->key2_left key2_y ->key2_right

pd.merge(df3, df4, on = 'key1')
# 	key1	key2_x	lval	key2_y	rval
# 0	foo	one	1	one	4
# 1	foo	one	1	one	5
# 2	foo	two	2	one	4
# 3	foo	two	2	one	5
# 4	bar	one	3	one	6
# 5	bar	one	3	two	7

pd.merge(df3, df4, on = 'key1', suffixes = ('_left', '_right'))
# key1	key2_left	lval	key2_right	rval
# 0	foo	one	1	one	4
# 1	foo	one	1	one	5
# 2	foo	two	2	one	4
# 3	foo	two	2	one	5
# 4	bar	one	3	one	6
# 5	bar	one	3	two	7

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [ ]:
#merge函数
# left
# right
# how
# on
# left_on
# right_on
# left_index
# right_index
# sort
# suffixes
# copy
# indicator

## 8.2.2 根据索引合并 Merging on Index

In [140]:
#left_index = True or right_index = True来表示索引需要用来作为合并的键：
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                     'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index =['a', 'b'])
left1
# 	key	value
# 0	a	0
# 1	b	1
# 2	a	2
# 3	a	3
# 4	b	4
# 5	c	5
right1
# # group_val
# # a	3.5
# # b	7.0

#类似于join的左边是key,右边是index
# left_on='key' 和 right_index=True：根据左侧DataFrame中的 'key' 列和右侧DataFrame的索引index进行匹配，将匹配的行合并在一起
pd.merge(left1, right1, left_on = 'key', right_index = True)
# 	key	value	group_val
# 0	a	0	3.5
# 2	a	2	3.5
# 3	a	3	3.5
# 1	b	1	7.0
# 4	b	4	7.0

#由于默认的合并方法是连接键相交，可以使用外连接进行合并：
pd.merge(left1, right1, left_on = 'key', right_index = True, how = 'outer')
# key	value	group_val
# 0	a	0	3.5
# 2	a	2	3.5
# 3	a	3	3.5
# 1	b	1	7.0
# 4	b	4	7.0
# 5	c	5	NaN



,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [141]:
#多层索引数据情况下，索引上连接是一个隐式的多键合并：
lefth = pd.DataFrame({'key1': ['Ohio','Ohio', 'Ohio', 'Nevada', 'Nevada'],
                     'key2': [2000, 2001, 2002, 2001, 2002],
                     'data': np.arange(5.)})
lefth
# 	key1	key2	data
# 0	Ohio	2000	0.0
# 1	Ohio	2001	1.0
# 2	Ohio	2002	2.0
# 3	Nevada	2001	3.0
# 4	Nevada	2002	4.0
righth = pd.DataFrame(np.arange(12).reshape((6,2)), 
                      index = [['Nevada','Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                              [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns = ['event1','event2'])
righth
# event1	event2
# Nevada	2001	0	1
# 2000	2	3
# Ohio	2000	4	5
# 2000	6	7
# 2001	8	9
# 2002	10	11

#join多个key：以列表的方式，指明合并所需多个列（使用how = 'outer'处理重复的索引值）：
pd.merge(lefth, righth, left_on = ['key1','key2'], right_index = True, how = 'outer')
# key1	key2	data	event1	event2
# 0	Ohio	2000	0.0	4.0	5.0
# 0	Ohio	2000	0.0	6.0	7.0
# 1	Ohio	2001	1.0	8.0	9.0
# 2	Ohio	2002	2.0	10.0	11.0
# 3	Nevada	2001	3.0	0.0	1.0
# 4	Nevada	2002	4.0	NaN	NaN
# 4	Nevada	2000	NaN	2.0	3.0

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [148]:
#使用两边的索引进行合并也可以：
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6]],
                    index = ['a', 'c', 'e'],
                    columns = ['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14]],
                     index = ['b', 'c', 'd', 'e'],
                     columns = ['Missouri', 'Alabama'])
left2
# Ohio	Nevada
# a	1.0	2.0
# c	3.0	4.0
# e	5.0	6.0
right2
# Missouri	Alabama
# b	7.0	8.0
# c	9.0	10.0
# d	11.0	12.0
# e	13.0	14.0

pd.merge(left2, right2, how = 'outer', left_index = True, right_index = True)
# 	Ohio	Nevada	Missouri	Alabama
# a	1.0	2.0	NaN	NaN
# b	NaN	NaN	7.0	8.0
# c	3.0	4.0	9.0	10.0
# d	NaN	NaN	11.0	12.0
# e	5.0	6.0	13.0	14.0

#data frame有一个方便的join实例方法，用于按照用于合并多个索引相同或相似，但没有重叠列的data frame.
left2.join(right2, how = 'outer')
# Ohio	Nevada	Missouri	Alabama
# a	1.0	2.0	NaN	NaN
# b	NaN	NaN	7.0	8.0
# c	3.0	4.0	9.0	10.0
# d	NaN	NaN	11.0	12.0
# e	5.0	6.0	13.0	14.0



,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [157]:
#data frame的join方法，进行连接键上的左连接，完全保留左边data frame的行索引。
left1.join(right1, on = 'key')
# 	key	value	group_val
# 0	a	0	3.5
# 1	b	1	7.0
# 2	a	2	3.5
# 3	a	3	3.5
# 4	b	4	7.0
# 5	c	5	NaN

#对于一些简单索引-索引合并，可以向join方法传入一个Data frame的列表，此法可以替代concat函数方法：
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                      index = ['a', 'c', 'e', 'f'],
                      columns = ['New York', 'Oregon'])

another
# New York	Oregon
# a	7.0	8.0
# c	9.0	10.0
# e	11.0	12.0
# f	16.0	17.0


#将三个DataFrame对象 left2、right2 和 another 进行连接操作。
#left2 是左侧的DataFrame，right2 和 another 是右侧的DataFrame。
#默认使用左连接（left join），即保留左侧DataFrame中的所有行，并将右侧DataFrame中匹配的行进行连接。
#连接后的结果将包含左侧DataFrame的所有列和右侧DataFrame的列。

left2.join([right2, another])
# 	Ohio	Nevada	Missouri	Alabama	New York	Oregon
# a	1.0	2.0	NaN	NaN	7.0	8.0
# c	3.0	4.0	9.0	10.0	9.0	10.0
# e	5.0	6.0	13.0	14.0	11.0	12.0

#加入how使其全连接。
left2.join([right2, another], how = 'outer')
# 	Ohio	Nevada	Missouri	Alabama	New York	Oregon
# a	1.0	2.0	NaN	NaN	7.0	8.0
# c	3.0	4.0	9.0	10.0	9.0	10.0
# e	5.0	6.0	13.0	14.0	11.0	12.0
# b	NaN	NaN	7.0	8.0	NaN	NaN
# d	NaN	NaN	11.0	12.0	NaN	NaN
# f	NaN	NaN	NaN	NaN	16.0	17.0

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


## 8.2.3 沿轴向连接 Concatenating Along an Axis

## 8.2.4 联合重叠数据 Combining Data with Overlap

# 8.3 重塑和透视 Reshaping and Pivoting

## 8.3.1 使用多层索引进行重塑 Reshaping with Hierarchical Indexing

## 8.3.2 将“长”透视为“宽” Pivoting “Long” to “Wide” Format

## 8.3.3 将“宽”透视为“长” 使用多层索引进行重塑 Pivoting “Wide” to “Long” Format

# 8.4 小结 Conclusion